In [1]:
from pynq import Overlay
from pynq.lib.video import *
import time

overlay = Overlay("/home/xilinx/base2.bit")

In [2]:
imgen_vdma = overlay.video.axi_vdma_0.readchannel
videoMode = common.VideoMode(640, 480, 24)
imgen_vdma.mode = videoMode
imgen_vdma.start()

hdmi_out = overlay.video.hdmi_out
hdmi_out._vdma = overlay.video.axi_vdma #Use the correct VDMA!
hdmi_out.configure(videoMode)
hdmi_out.start()

pixgen = overlay.pixel_generator_0

In [10]:
start = time.time()
for i in range(0, 9599, 16):
    pixgen.register_map.gp0 = i
    frame = imgen_vdma.readframe()
    hdmi_out.writeframe(frame)

end = time.time()
framerate = 600/(end-start)
print(framerate)

59.606307133226146
